In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Android_Activity/raw_activity_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['uid', 'date', 'stationary_duration_morning',
       'walking_duration_morning', 'running_duration_morning',
       'unknown_duration_morning', 'stationary_duration_afternoon',
       'walking_duration_afternoon', 'running_duration_afternoon',
       'unknown_duration_afternoon', 'stationary_duration_evening',
       'walking_duration_evening', 'running_duration_evening',
       'unknown_duration_evening', 'stationary_duration_night',
       'walking_duration_night', 'running_duration_night',
       'unknown_duration_night', 'stress_ratings'],
      dtype='object')


In [4]:
df.isnull().sum()

uid                              0
date                             0
stationary_duration_morning      0
walking_duration_morning         0
running_duration_morning         0
unknown_duration_morning         0
stationary_duration_afternoon    0
walking_duration_afternoon       0
running_duration_afternoon       0
unknown_duration_afternoon       0
stationary_duration_evening      0
walking_duration_evening         0
running_duration_evening         0
unknown_duration_evening         0
stationary_duration_night        0
walking_duration_night           0
running_duration_night           0
unknown_duration_night           0
stress_ratings                   0
dtype: int64

In [5]:
df = df.dropna()
print(len(df))

1216


In [6]:
df.columns

Index(['uid', 'date', 'stationary_duration_morning',
       'walking_duration_morning', 'running_duration_morning',
       'unknown_duration_morning', 'stationary_duration_afternoon',
       'walking_duration_afternoon', 'running_duration_afternoon',
       'unknown_duration_afternoon', 'stationary_duration_evening',
       'walking_duration_evening', 'running_duration_evening',
       'unknown_duration_evening', 'stationary_duration_night',
       'walking_duration_night', 'running_duration_night',
       'unknown_duration_night', 'stress_ratings'],
      dtype='object')

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    484
low stress       366
high stress      366
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

logo = LeaveOneGroupOut()

In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
print(num_splits)

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print('X and y train shapes: ')
        print(X_train.shape)
        print(y_train.shape)

        train_class_counts = np.bincount(y_train)
        print(f"Train set - Class 0 (Low Stress): {train_class_counts[0]}, "
              f"Class 1 (High Stress): {train_class_counts[1] if len(train_class_counts) > 1 else 0}")

        test_class_counts = np.bincount(y_test)
        print(f"Test set - Class 0 (Low Stress): {test_class_counts[0]}, "
              f"Class 1 (High Stress): {test_class_counts[1] if len(test_class_counts) > 1 else 0}")

        print(f"Test set size: {len(y_test)}")
        print(f"Train set shape: {X_train.shape}")
        print(f"Test set shape: {X_test.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(X_train_normalized.shape)
        print(X_test_normalized.shape)

        # Define the FCN model
        model_fcn_binary_lh = tf.keras.models.Sequential([
            tf.keras.layers.Dense(57, activation='relu', input_dim=X_train_normalized.shape[1]),
            tf.keras.layers.Dropout(0.35),
            tf.keras.layers.Dense(35, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(3, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
        ])
        print('Model Defined')

        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model_fcn_binary_lh.compile(
            optimizer=custom_optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']  
        )

        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(enumerate(class_weights))

        print('Class Weight: ', class_weight_dict)

        model_fcn_binary_lh.fit(
            X_train_normalized, y_train,
            epochs=100,
            batch_size=32,
            class_weight=class_weight_dict,
            verbose=1  
        )

        thresholds = np.arange(0.01, 1.0, 0.01)
        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        best_threshold = 0.0
        best_metric = 0.0

        for threshold in thresholds:
            y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
            metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)

            if metric_value > best_metric:
                best_metric = metric_value
                best_threshold = threshold

        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)  
            if auc_score < 0.5:
                print(f"<0.5 AUC Score: {auc_score}")
                print("----------------------------------------")
            else:
                print(f"AUC Score: {auc_score}")

        else:
            auc_scores.append(None)  
            print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc) 
        best_thresholds.append(best_threshold)  
        print(f"Balanced Accuracy: {balanced_acc}")

        pbar.update(1)

46


LOSO CV Progress:   0%|                                    | 0/46 [00:00<?]   0%WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


X and y train shapes: 
(707, 16)
(707,)
Train set - Class 0 (Low Stress): 354, Class 1 (High Stress): 353
Test set - Class 0 (Low Stress): 12, Class 1 (High Stress): 13
Test set size: 25
Train set shape: (707, 16)
Test set shape: (25, 16)
(707, 16)
(25, 16)
Model Defined
Class Weight:  {0: 0.998587570621469, 1: 1.001416430594901}
Epoch 1/100


2024-10-05 10:17:20.504576: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


23/23 [==============================] - 0s 541us/step - loss: 0.6913 - accuracy: 0.5417
Epoch 2/100
23/23 [==============================] - 0s 437us/step - loss: 0.6956 - accuracy: 0.5078
Epoch 3/100
23/23 [==============================] - 0s 409us/step - loss: 0.6960 - accuracy: 0.5177
Epoch 4/100
23/23 [==============================] - 0s 409us/step - loss: 0.6925 - accuracy: 0.5417
Epoch 5/100
23/23 [==============================] - 0s 445us/step - loss: 0.6891 - accuracy: 0.5559
Epoch 6/100
23/23 [==============================] - 0s 474us/step - loss: 0.6940 - accuracy: 0.5318
Epoch 7/100
23/23 [==============================] - 0s 495us/step - loss: 0.6894 - accuracy: 0.5728
Epoch 8/100
23/23 [==============================] - 0s 502us/step - loss: 0.6865 - accuracy: 0.5644
Epoch 9/100
23/23 [==============================] - 0s 466us/step - loss: 0.6882 - accuracy: 0.5474
Epoch 10/100
23/23 [==============================] - 0s 493us/step - loss: 0.6870 - accuracy: 0.5205
E

...

LOSO CV Progress:  98%|█████████████████████████████▎ | 45/46 [01:04<00:01]  98%

<0.5 AUC Score: 0.475
----------------------------------------
Balanced Accuracy: 0.5083333333333333
X and y train shapes: 
(678, 16)
(678,)
Train set - Class 0 (Low Stress): 323, Class 1 (High Stress): 355
Test set - Class 0 (Low Stress): 43, Class 1 (High Stress): 11
Test set size: 54
Train set shape: (678, 16)
Test set shape: (54, 16)
(678, 16)
(54, 16)


Model Defined
Class Weight:  {0: 1.0495356037151702, 1: 0.9549295774647887}
Epoch 1/100
22/22 [==============================] - 0s 534us/step - loss: 0.7188 - accuracy: 0.4720
Epoch 2/100
22/22 [==============================] - 0s 445us/step - loss: 0.6926 - accuracy: 0.5324
Epoch 3/100
22/22 [==============================] - 0s 427us/step - loss: 0.6998 - accuracy: 0.5324
Epoch 4/100
22/22 [==============================] - 0s 416us/step - loss: 0.6892 - accuracy: 0.5487
Epoch 5/100
22/22 [==============================] - 0s 442us/step - loss: 0.6896 - accuracy: 0.5369
Epoch 6/100
22/22 [==============================] - 0s 443us/step - loss: 0.6879 - accuracy: 0.5369
Epoch 7/100
22/22 [==============================] - 0s 445us/step - loss: 0.6907 - accuracy: 0.5472
Epoch 8/100
22/22 [==============================] - 0s 446us/step - loss: 0.6898 - accuracy: 0.5310
Epoch 9/100
22/22 [==============================] - 0s 445us/step - loss: 0.6892 - accuracy: 0.5236
Epoch 10/100
22

LOSO CV Progress: 100%|██████████████████████████████ | 46/46 [01:05<00:00] 100%

AUC Score: 0.5221987315010571
Balanced Accuracy: 0.587737843551797


In [13]:
print('MLP: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

MLP: 
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.5625, 25th Percentile: 0.4482142857142857, 75th Percentile: 0.6886817226890756
Balanced Accuracy - Median: 0.686186974789916, 25th Percentile: 0.5885306553911205, 75th Percentile: 0.765625
